# IDScalarWaveNRPy: An Einstein Toolkit Initial Data Thorn for the Scalar Wave Equation

## Author: Zach Etienne
### Formatting improvements courtesy Brandon Clark

### NRPy+ Source Code for this module: [ScalarWave/InitialData_PlaneWave.py](../edit/ScalarWave/InitialData_PlaneWave.py)[\[**tutorial**\]](Tutorial-ScalarWave.ipynb) Contructs the SymPy expressions for plane-wave initial data

## Introduction:
In this part of the tutorial, we will construct an Einstein Toolkit (ETK) thorn (module) that will set up *initial data* for the scalar wave initial value problem. In a [previous tutorial module](Tutorial-ScalarWave.ipynb), we used NRPy+ to contruct the SymPy expressions for plane-wave initial data. This thorn is largely based on and should function similarly to the $\text{IDScalarWaveC}$ thorn included in the Einstein Toolkit (ETK) $\text{CactusWave}$ arrangement.

We will construct this thorn in two steps.

1. Call on NRPy+ to convert the SymPy expressions for the initial data into one C-code kernel.
1. Write the C code and linkages to the Einstein Toolkit infrastructure (i.e., the .ccl files) to complete this Einstein Toolkit module.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$ 

This module is organized as follows

1. [Step 1](#initializenrpy): Call on NRPy+ to convert the SymPy expression for the scalar wave initial data into a C-code kernel
1. [Step 2](#einstein): Interfacing with the Einstein Toolkit
    1. [Step 2.a](#einstein_c): Constructing the Einstein Toolkit C-code calling functions that include the C code kernels
    1. [Step 2.b](#einstein_ccl): CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure
    1. [Step 2.c](#einstein_list): Add the C code to the Einstein Toolkit compilation list
1. [Step 3](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='initializenrpy'></a>

# Step 1: Call on NRPy+ to convert the SymPy expression for the scalar wave initial data into a C-code kernel \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

After importing the core modules, we will set $\text{GridFuncMemAccess}$ to $\text{ETK}$. SymPy expressions for plane wave initial data are written inside [ScalarWave/InitialData_PlaneWave.py](../edit/ScalarWave/InitialData_PlaneWave.py), and we simply import them for use here.

In [1]:
# Step 1: Call on NRPy+ to convert the SymPy expression 
#         for the scalar wave initial data into a C-code kernel

# Step 1a: Import needed NRPy+ core modules:
import NRPy_param_funcs as par
import indexedexp as ixp
import grid as gri
import finite_difference as fin
from outputC import *
import loop

# Step 1b: This is an Einstein Toolkit (ETK) thorn. Here we
#          tell NRPy+ that gridfunction memory access will 
#          therefore be in the "ETK" style.
par.set_parval_from_str("grid::GridFuncMemAccess","ETK")

# Step 1c: Call the InitialData_PlaneWave() function from within the
#          ScalarWave/InitialData_PlaneWave.py module.
import ScalarWave.InitialData_PlaneWave as swid

# Step 1d: Within the ETK, the 3D gridfunctions x, y, and z store the
#         Cartesian grid coordinates. Setting the gri.xx[] arrays
#         to point to these gridfunctions forces NRPy+ to treat
#         the Cartesian coordinate gridfunctions properly --
#         reading them from memory as needed.
x,y,z = gri.register_gridfunctions("AUX",["x","y","z"])
gri.xx[0] = x
gri.xx[1] = y
gri.xx[2] = z

# Step 1e: Set up the plane wave initial data. This sets uu_ID and vv_ID.
swid.InitialData_PlaneWave()

# Step 1f: Register uu and vv gridfunctions so they can be written to by NRPy.
uu,vv = gri.register_gridfunctions("EVOL",["uu","vv"])

# Step 1g: Set the uu and vv gridfunctions to the uu_ID & vv_ID variables 
#         defined by InitialData_PlaneWave().
uu = swid.uu_ID
vv = swid.vv_ID

# Step 1h: Create the C code output kernel.
scalar_PWID_to_print = [\
                        lhrh(lhs=gri.gfaccess("out_gfs","uu"),rhs=uu),\
                        lhrh(lhs=gri.gfaccess("out_gfs","vv"),rhs=vv),]
scalar_PWID_CcodeKernel = fin.FD_outputC("returnstring",scalar_PWID_to_print)

scalar_PWID_looped = loop.loop(["i2","i1","i0"],["1","1","1"],["cctk_lsh[2]-1","cctk_lsh[1]-1","cctk_lsh[0]-1"],\
                               ["1","1","1"],["#pragma omp parallel for","",""],"",\
                               scalar_PWID_CcodeKernel.replace("time","cctk_time"))

# Step 1i: Create directories for the thorn if they don't exist.
!mkdir IDScalarWaveNRPy     2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.
!mkdir IDScalarWaveNRPy/src 2>/dev/null # 2>/dev/null: Don't throw an error if the directory already exists.

# Step 1j: Write the C code kernel to file.
with open("IDScalarWaveNRPy/src/ScalarWave_PWID.h", "w") as file:
    file.write(str(scalar_PWID_looped))


<a id='einstein'></a>

# Step 2: Interfacing with the Einstein Toolkit \[Back to [top](#toc)\]
$$\label{einstein}$$


<a id='einstein_c'></a>

## Step 2.a: Constructing the Einstein Toolkit C-code calling functions that include the C code kernels \[Back to [top](#toc)\]
$$\label{einstein_c}$$

We will write another C file with the functions we need here.

In [2]:
%%writefile IDScalarWaveNRPy/src/InitialData.c
#include <math.h>
#include <stdio.h>

#include "cctk.h"
#include "cctk_Parameters.h"
#include "cctk_Arguments.h"

void IDScalarWaveNRPy_param_check(CCTK_ARGUMENTS) {
  DECLARE_CCTK_ARGUMENTS;
  DECLARE_CCTK_PARAMETERS;
  
  if (kk0 == 0 && kk1 == 0 && kk2 == 0) {
     CCTK_WARN(0,"kk0==kk1==kk2==0: Zero wave vector cannot be normalized. Set one of the kk's to be != 0.");
  }
}

void IDScalarWaveNRPy_InitialData(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS
  DECLARE_CCTK_PARAMETERS
  
  const CCTK_REAL *xGF = x;
  const CCTK_REAL *yGF = y;
  const CCTK_REAL *zGF = z;
#include "ScalarWave_PWID.h"
}

Overwriting IDScalarWaveNRPy/src/InitialData.c


<a id='einstein_ccl'></a>

## Step 2. b: CCL files - Define how this module interacts and interfaces with the larger Einstein Toolkit infrastructure \[Back to [top](#toc)\]
$$\label{einstein_ccl}$$

Writing a module ("thorn") within the Einstein Toolkit requires that three "ccl" files be constructed, all in the root directory of the thorn:

1. $\text{interface.ccl}$: defines the gridfunction groups needed, and provides keywords denoting what this thorn provides and what it should inherit from other thorns. Specifically, this file governs the interaction between this thorn and others; more information can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-260000C2.2). 
With "implements", we give our thorn its unique name. By "inheriting" other thorns, we tell the Toolkit that we will rely on variables that exist and are declared "public" within those functions.

In [3]:
%%writefile IDScalarWaveNRPy/interface.ccl

implements: IDScalarWaveNRPy
    
inherits: WaveToyNRPy grid


Overwriting IDScalarWaveNRPy/interface.ccl


2. $\text{param.ccl}$: specifies free parameters within the thorn, enabling them to be set at runtime. It is required to provide allowed ranges and default values for each parameter. More information on this file's syntax can be found in the [official Einstein Toolkit documentation](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-265000C2.3).

In [4]:
%%writefile IDScalarWaveNRPy/param.ccl
shares: grid

USES KEYWORD type

shares: WaveToyNRPy
    
USES REAL wavespeed

restricted:
CCTK_KEYWORD initial_data "Type of initial data"
{
  "plane"      :: "Plane wave"
} "plane"

restricted:
CCTK_REAL kk0 "The wave number in the x-direction"
{
 *:* :: "No restriction"
} 4.0

restricted:
CCTK_REAL kk1 "The wave number in the y-direction"
{
 *:* :: "No restriction"
} 0.0

restricted:
CCTK_REAL kk2 "The wave number in the z-direction"
{
 *:* :: "No restriction"
} 0.0


Overwriting IDScalarWaveNRPy/param.ccl


3. $\text{schedule.ccl}$: allocates storage for gridfunctions, defines how the thorn's functions should be scheduled in a broader simulation, and specifies the regions of memory written to or read from gridfunctions. $\text{schedule.ccl}$'s official documentation may be found [here](http://cactuscode.org/documentation/referencemanual/ReferenceManualch8.html#x12-268000C2.4). 

We specify here the standardized ETK "scheduling bins" in which we want each of our thorn's functions to run.

In [5]:
%%writefile IDScalarWaveNRPy/schedule.ccl

schedule IDScalarWaveNRPy_param_check at CCTK_PARAMCHECK
{
  LANG: C
  OPTIONS: global
} "Check sanity of parameters"

schedule IDScalarWaveNRPy_InitialData at CCTK_INITIAL as WaveToy_InitialData
{
  STORAGE:       WaveToyNRPy::scalar_fields[3]
  LANG:          C
  READS: grid::x(Everywhere)
  READS: grid::y(Everywhere)
  READS: grid::y(Everywhere)
  WRITES: uuGF(Everywhere)
  WRITES: vvGF(Everywhere)
} "Initial data for 3D wave equation"


Overwriting IDScalarWaveNRPy/schedule.ccl


<a id='einstein_list'></a>

## Step 2.c: Add the C code to the Einstein Toolkit compilation list \[Back to [top](#toc)\]
$$\label{einstein_list}$$

We will also need $\text{make.code.defn}$, which indicates the list of files that need to be compiled. This thorn only has the one C file to compile.

In [6]:
%%writefile IDScalarWaveNRPy/src/make.code.defn

SRCS = InitialData.c

Overwriting IDScalarWaveNRPy/src/make.code.defn


<a id='latex_pdf_output'></a>

# Step 3: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-ETK_thorn-IDScalarWaveNRPy.pdf](Tutorial-ETK_thorn-IDScalarWaveNRPy.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [7]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-IDScalarWaveNRPy.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-IDScalarWaveNRPy.tex
!pdflatex -interaction=batchmode Tutorial-ETK_thorn-IDScalarWaveNRPy.tex
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-ETK_thorn-IDScalarWaveNRPy.ipynb to latex
[NbConvertApp] Writing 43836 bytes to Tutorial-ETK_thorn-IDScalarWaveNRPy.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
